# Введение в методы сбора и обработки данных
## Добашнее задание к четвертому уроку "Парсинг HTML (Xpath/BeautifulSoup)"

### Задача 1
С помощью BeautifulSoup спарсить новости с https://news.yandex.ru по своему региону.

* Заголовок
* Краткое описание
* Ссылка на новость

#### Решение

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint

Скачаем контент со страницы новостей

In [2]:
def fake_get_site_content():
    return open('ya_news.html', 'r').read()

def get_site_content():
    return requests.get('https://news.yandex.ru/Moscow_and_Moscow_Oblast').text

In [3]:
content = get_site_content()

Спарсим контент и сформируем из него датасет

In [46]:
def parse_yandex_news(content):
    soup = bs(content, 'html.parser')
    tag_class_news = [{'class': 'story__topic'}, 
                      {'class': 'story story_view_normal'},
                      {'class': 'story story_view_short'}, 
                      {'class': 'story story_view_with-left-image'},
                      {'class': 'story story_view_normal story_noimage'}]

    divs_list = []
    for tag in tag_class_news:
        divs_list.extend(soup.find_all('div', tag))
        
    return divs_list
    
def get_df_from_divs(divs):
    """
    Возвращает датасет из спарсенных новостей.
    Признаки датасета: Заголовок, Краткое описание, Ссылка
    """
    news = []
    for div in divs:
        # Заголовок новости
        title = div.find('a', attrs={'class': "link link_theme_black i-bem"}).text
        
        # Краткое описание новости
        try:
            content = div.find('div', attrs={'class': "story__text"}).text
        except:
            content = None
        
        # Ссылка на новость
        href = div.find('a', attrs={'class': "link link_theme_black i-bem"})['href'].replace(' ', '')
        if href.startswith('/story/'):
            href = f'https://news.yandex.ru{href}'
        
        news.append({
            'title': title,
            'content': content,
            'href': href
        })

    return pd.DataFrame(news)[['title', 'content', 'href']]

In [47]:
divs = parse_yandex_news(content)
df = get_df_from_divs(divs)

In [48]:
df

,title,content,href
0,"Мальчик Серёжа, найденный в Щёлково, обрёл нов...",Около года назад в Щёлкове на улице Комарова б...,https://news.yandex.ru/story/Malchik_Seryozha_...
1,Электрички Горьковского направления МЖД следую...,None,https://news.yandex.ru/story/EHlektrichki_Gork...
2,Конкурс «Аварийный район» для военнослужащих о...,None,https://yandex.ru/sport/story/Konkurs_Avarijny...
3,Два человека погибли при пожаре в квартире в С...,None,https://news.yandex.ru/story/Dva_cheloveka_pog...
4,Погода постепенно налаживается в Люберцах,None,https://news.yandex.ru/story/Pogoda_postepenno...
5,День физкультурника в Химках отметили с конкур...,"Организаторы отметили, что в этом году впервые...",https://yandex.ru/sport/story/Den_fizkulturnik...
6,День молодёжи отмечают в Королёве,"Несмотря на плохую погоду, на «Вымпел» пришло ...",https://news.yandex.ru/story/Den_molodyozhi_ot...
7,Мастер‑классы по спортивным танцам пройдут в п...,Танцплощадки откроются 10 августа в центрально...,https://yandex.ru/sport/story/Masterklassy_po_...
8,В Серпухове построят ФОК с бассейном,"Согласно проектной документации, в здании пред...",https://news.yandex.ru/story/V_Serpukhove_post...
9,Новый корпус школы в Балашихе построят в 2020 ...,Школьный корпус на 400 мест будет располагатьс...,https://news.yandex.ru/story/Novyj_korpus_shko...


### Задача 2  *
* Разбить новости по категориям
* Расположить в хронологическом порядке

#### Решение
Задача сводится в добавлению таких полей как `тема` `дата`. На ранжирование по дате у меня, к сожалению, времени нет, а вот сгруппировать по темам - можно

In [58]:
def get_df_from_divs_extended(divs):
    """
    Возвращает датасет из спарсенных новостей в хронологическом порядке.
    Признаки датасета: Тема, Дата, Заголовок, Краткое описание, Ссылка
    """
    news = []
    for div in divs:
        theme = div.find('a').text
        
        # Заголовок новости
        title = div.find('a', attrs={'class': "link link_theme_black i-bem"}).text
        
        # Краткое описание новости
        try:
            content = div.find('div', attrs={'class': "story__text"}).text
        except:
            content = None
        
        # Ссылка на новость
        href = div.find('a', attrs={'class': "link link_theme_black i-bem"})['href'].replace(' ', '')
        if href.startswith('/story/'):
            href = f'https://news.yandex.ru{href}'
        
        news.append({
            'theme': theme,
            'title': title,
            'content': content,
            'href': href
        })

    return pd.DataFrame(news)[['theme', 'title', 'content', 'href']].sort_values(by=['theme'])

In [59]:
df_ext = get_df_from_divs_extended(divs)

In [60]:
df_ext

,theme,title,content,href
12,ЖКХ,Готовность многоквартирных домов к зиме провер...,Готовность многоквартирных домов к зимнему ото...,https://news.yandex.ru/story/Gotovnost_mnogokv...
10,ЖКХ,На региональных дорогах в Подмосковье ликвидир...,Дорожные службы ликвидируют последствия непого...,https://news.yandex.ru/story/Na_regionalnykh_d...
0,Общество,"Мальчик Серёжа, найденный в Щёлково, обрёл нов...",Около года назад в Щёлкове на улице Комарова б...,https://news.yandex.ru/story/Malchik_Seryozha_...
26,Общество,Две с половиной тысячи человек уже зачислены в...,Уже порядка двух с половиной тысяч человек зач...,https://news.yandex.ru/story/Dve_s_polovinoj_t...
25,Общество,Уважаемые работники и ветераны строительной от...,Вы обеспечиваете надежную работу строительного...,https://news.yandex.ru/story/Uvazhaemye_rabotn...
24,Общество,Воробьев поздравил жителей Озер и Зарайска с Д...,Губернатор Московской области Андрей Воробьев ...,https://news.yandex.ru/story/Vorobev_pozdravil...
23,Общество,Дата открытия долгожданного боулинг-клуба в Од...,Пресс-служба Одинцовского подворья сообщила да...,https://news.yandex.ru/story/Data_otkrytiya_do...
22,Общество,Родителям напоминают про безопасность детей на...,"Необходимо объяснить ребенку, почему не следуе...",https://news.yandex.ru/story/Roditelyam_napomi...
19,Общество,608 щёлковцам планируется вручить памятные под...,None,https://news.yandex.ru/story/608_shhyolkovcam_...
13,Общество,Лучших строителей наградили в Московской области,Лучших строителей наградили в Московской облас...,https://news.yandex.ru/story/Luchshikh_stroite...
